In [2]:
import pandas as pd
import json
import csv
import cplex
import time

In [3]:
def listar(x):
    rawData = []
    start = 0
    for i in range(len(x)):
        if x[i] == ' ' or x[i] == '\n':
            rawData.append(x[start:i])
            start = i + 1
    return rawData

def print_matrix(matrix):
    for row in matrix:
        for element in row:
            print(element, end=" ")
        print()
        
def ler_grafo(u):
    arquive = open(u, 'r')
    content = arquive.read()
    rawData = []
    rawData = listar(content)

    vertices, arestas = [int(v) for v in rawData[:2]]
    edgeData = [json.loads(edge) for edge in rawData[2:]]
    print(str(vertices) + ' vertices, ' + str(arestas) + ' arestas, ')
    # print(edgeData)
    m1, m2 = [], []
    for i in range(vertices):
        m1.append([0]*vertices)
        m2.append([])
    
    for i in range(0,len(edgeData)-1, 2):
        m1[edgeData[i]][edgeData[i+1]] = 1
        m1[edgeData[i+1]][edgeData[i]] = 1

    if len(m1) != 0:
        for i in  range(len(m1)):
            for j in range(len(m1[i])):
                if m1[i][j] == 1:
                    m2[i].append(j+1)
    # print_matrix(m1)
    # print()
    # print_matrix(m2)
    return m2,vertices,arestas



In [4]:
def resolve_modelo(m2,vertices,arestas,k, timethresh):
    
    cpx = cplex.Cplex()

    y = cpx.variables.add(obj=[0] * vertices,
                             lb=[0] * vertices, ub=[1] * vertices,
                             types=['B'] * vertices,
                             names=['y_%d' % (p) for p in range(1,vertices+1)])
    h = cpx.variables.add(obj=[1] * vertices,
                            lb=[0] * vertices, ub=[1] * vertices,
                            types=['B'] * vertices,
                            names=['h_%d' % (z) for z in range(1,vertices+1)])

    #Restrição 2
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([y[i] for i in range(vertices)], [1.0]*vertices)],
        senses=['E'],
        rhs= [k])


    #Restrição 3
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]], [1.0,-1.0])
                  for i in range(vertices)],
        senses=['L'] * vertices,
        rhs=[0.0] * vertices)
    
    
    #Restrição 4
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[j-1]], [1.0,-1.0])
                  for i in range(vertices) for j in m2[i]],
        senses=['L'] * (arestas*2),
        rhs=[0.0] * (arestas*2))
    
    
    #Restrição 5
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]]+[y[j-1] for j in m2[i]], [1.0,-1.0]+[-1.0]*len(m2[i]))
                  for i in range(vertices)],
        senses=['G'] * vertices,
        rhs=[-len(m2[i]) for i in range(vertices)])

    
    cpx.parameters.threads.set(1)
    cpx.parameters.timelimit.set(timethresh)
    cpx.objective.set_sense(cpx.objective.sense.maximize)
    
    cpx.write('model.lp')
    start_time = time.time()
    cpx.solve()
    my_time = time.time() - start_time
    status = cpx.solution.get_status()
    # if status != 101:
    #     relaxation = [cpx.solution.MIP.get_best_objective(),
    #                     cpx.solution.MIP.get_cutoff,
    #                     # cpx.solution.MIP.get_incubent_node(),
    #                     # cpx.solution.MIP.get_mip_relative_gap(),
    #                     cpx.solution.MIP.get_num_cuts(cpx.solution.MIP.cut_type.cover),
    #                     cpx.solution.MIP.get_subproblem_status()]
    # else:
    relaxation = []

    return status, cpx.solution.get_objective_value(), cpx.solution.MIP.get_best_objective(), ','.join(relaxation), cpx.solution.get_objective_value(), my_time


In [ ]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = {
    'BA': False,
    'bipartite': False,
    'lollipop': False,
    'star': False,
    'cubic': False
}
time_threshold = 900
for log_n in range(2,15):
    n = 2**log_n
    for graphClass in ["star", "cubic", "bipartite", "BA", "lollipop"]:
        for k in range(1,4):
            k = 2**(log_n + k - 4)
            filename = f'graphs/{graphClass}/{graphClass}_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            if not deuruim[graphClass]:
                m2,v,e = ler_grafo(filename)
                status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
                if status != 101:
                    deuruim[graphClass] = True
                results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
                print(f'------------------------>{filename}_k:{k} feito.')

                index += 1
            results.to_csv(f'./results4.csv')
print("Fim.")


In [ ]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = 0
time_threshold = 2800
if deuruim < 4:
    for log_n in range(2,15):
        n = 2**log_n
        kStep = int(n/20)
        if kStep == 0:
            kStep = 1
        for k in range(0,n,kStep):
            filename = f'graphs/BA/BA_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            m2,v,e = ler_grafo(filename)
            status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
            if status != 101:
                deuruim += 1
            results.loc[index] = [f'{filename}_{k}', "BA", n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
            print(f'------------------------>{filename}_k:{k} feito.')
            index += 1
            results.to_csv(f'./results5.csv')
print("Fim.")


In [5]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = {
    'BA': False,
    'bipartite': False,
    'lollipop': False,
    'star': False,
    'cubic': False
}
time_threshold = 3600
for graphClass in ["BA", "lollipop"]:
    for log_n in range(2,15):
        n = 2**log_n
        kStep = int(n/20)
        if kStep == 0:
            kStep = 1
        for k in range(0,n,kStep):
            filename = f'graphs/{graphClass}/{graphClass}_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            if not deuruim[graphClass]:
                m2,v,e = ler_grafo(filename)
                status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
                if status != 101:
                    deuruim[graphClass] = True
                results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
                print(f'------------------------>{filename}_k:{k} feito.')
                index += 1
                results.to_csv(f'./results7.csv')
print("Fim.")


Default row names c1, c2 ... being created.


Começando graphs/BA/BA_4.txt_k:0.------------------------>
4 vertices, 4 arestas, 
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 17 rows and 8 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.01 ticks)


Default row names c1, c2 ... being created.


------------------------>graphs/BA/BA_4.txt_k:0 feito.
Começando graphs/BA/BA_4.txt_k:1.------------------------>
4 vertices, 4 arestas, 
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 15 rows and 7 columns.
MIP Presolve modified 14 coefficients.
Aggregator did 1 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.04 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.04 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.04 ticks)


Default row names c1, c2 ... being created.


...

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Tried aggregator 1 time.
Reduced MIP has 1053697 rows, 4096 columns, and 3159040 nonzeros.
Reduced MIP has 4096 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 19.80 sec. (7140.75 ticks)
Found incumbent of value 1427.000000 after 20.55 sec. (7533.08 ticks)
Probing time = 0.07 sec. (8.42 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Elapsed time for symmetry detection = 61.35 sec. (10000.38 ticks)
Elapsed time for symmetry detection = 129.77 sec. (20000.70 ticks)
Elapsed time for symmetry detection = 202.43 sec. (30000.95 ticks)
Elapsed time for symmetry detection = 271.25 sec. (40001.12 ticks)
Elapsed time for symmetry detection = 328.17 sec. (50001.30 ticks)
Elapsed time for symmetry detection = 390.57 sec. (60001.54 ticks)
Found 2.800031e+5273 symmetri

Default row names c1, c2 ... being created.


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Tried aggregator 1 time.
Reduced MIP has 1053697 rows, 4096 columns, and 3159040 nonzeros.
Reduced MIP has 4096 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 16.78 sec. (7140.75 ticks)
Found incumbent of value 1529.000000 after 17.70 sec. (7533.08 ticks)
Probing time = 0.10 sec. (8.42 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Elapsed time for symmetry detection = 64.90 sec. (10000.38 ticks)
Elapsed time for symmetry detection = 135.57 sec. (20000.70 ticks)
Elapsed time for symmetry detection = 202.14 sec. (30000.95 ticks)
Elapsed time for symmetry detection = 262.46 sec. (40001.12 ticks)
Elapsed time for symmetry detection = 322.55 sec. (50001.30 ticks)
Elapsed time for symmetry detection = 381.15 sec. (60001.54 ticks)
Found 2.800031e+5273 symmetri

Default row names c1, c2 ... being created.


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Tried aggregator 1 time.
Reduced MIP has 1053697 rows, 4096 columns, and 3159040 nonzeros.
Reduced MIP has 4096 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 15.75 sec. (7140.75 ticks)
Found incumbent of value 1631.000000 after 16.43 sec. (7533.09 ticks)
Probing time = 0.07 sec. (8.42 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Elapsed time for symmetry detection = 61.00 sec. (10000.38 ticks)
Elapsed time for symmetry detection = 128.92 sec. (20000.70 ticks)
Elapsed time for symmetry detection = 201.89 sec. (30000.95 ticks)
Elapsed time for symmetry detection = 274.50 sec. (40001.12 ticks)


In [ ]:
results

In [ ]:
results